<a href="https://colab.research.google.com/github/Wang-Kevin900/DataVisualization/blob/main/EsercizioReti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup libraries

In [4]:
!pip install bokeh
!pip install altair
!pip install plotly
!pip install pandas
!pip install datetime

     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 245kB 25.3MB/s 


# Get data

Importing the various library needed to the program, importing the txt sheet where all the data are (format: date - hour - value) (as long there's new line to indicate that a set of value is done, the program will work no matter how the data is placed in the datasheet, it is fault resistant) and lastly creating all the needed variable

In [5]:
from google.colab import files
from datetime import datetime

# get the data from the text file
file_data = files.upload()
# trasform the data got from the txt file to list, 
# so that in the next FOR cycle for trasforming the data is easier to read letter per letter
string_datatext_value = list(file_data.values());

Saving Dati1.txt to Dati1.txt


# Trasform Data

Preparing the needed variable to trasform the data read from the datasheet, to lists

In [6]:
# variable that indicate the number of symbols that compose a date&time
date_format = 10
time_format = 8

# variable that will store every symbol read from the datasheet
buffer_of_data = ""

# variable that will store any complete data with any of the following format (date, time, float)
single_string_list_of_data = []

# once a single line of data (couple of data) is done being stored, 
# divide them (because later they will be trasformed in a pandas dataframe)
list_of_datetime = []
list_of_float = []

# divide a single line of data of the datasheet in 3 section, where each section has the following meaning,
# Date, Time, Float; what this variable do, is basically acting as index 
# to indicate wich section we are currently reading in the datasheet
single_index_data_value = 0

Rearranging all the data in 2 list (datatime and float), while also doing the casting from string to the needed type

In [7]:
from datetime import datetime

# go through all the data, and return a list of the various couples of data
for i in string_datatext_value[0]:
  # 32 is a value ascii that means "Space"
  # 9 is a value ascii that means "Tab"
  # 13 is a value ascii that means "Carriage Return"
  # 10 is a value ascii that means "New Line"
  # Ignore all the "special symbols", and just get all the "interesting" data
  if i != 32 and i != 9 and i != 13:
    # read from string of data (taken from the datasheet) and store it
    buffer_of_data = buffer_of_data + chr(i);

    # if we are currently in the first section (date section)...
    if single_index_data_value == 0:
      # if the lenght of the collected data reached the date format, that means that we are done with the date section,
      # also it means that the casting is possible
      if len(buffer_of_data) == date_format:
        single_string_list_of_data.append(datetime.strptime(buffer_of_data, "%d-%m-%Y").date())

        # reset the buffer and pass to the next section
        buffer_of_data = ""
        single_index_data_value+=1

    # if we are currently in the second section (time section)...
    elif single_index_data_value == 1:
      # if the lenght of the collected data reached the time format, that means that we are done with the time section,
      # also it means that the casting is possible
      if len(buffer_of_data) == time_format:
        single_string_list_of_data.append(datetime.strptime(buffer_of_data, "%H:%M:%S").time())

        # reset the buffer and pass to the next section
        buffer_of_data = ""
        single_index_data_value+=1

    # if we are currently in the third section (float section)...
    elif single_index_data_value == 2:
      # if we detect a 10 (in ASCII means "new line"), that means we're done reading a single line of data
      # and that we are done to also reading a single couple of data
      if i == 10:
        # Add the string to the string of data while also,
        # Converting the value from string to float and replacing the "," in "." so that the casting is possible
        single_string_list_of_data.append(float(buffer_of_data.replace(",", ".")))

        # divide the couple of data into 2 different lists to prepare them to be converted into pandas dataframe
        list_of_datetime.append(datetime.combine(single_string_list_of_data[0], single_string_list_of_data[1]))
        list_of_float.append(single_string_list_of_data[2])

        # reset the buffer and the single string of data list and return to the first section to read more data of other lines
        buffer_of_data = ""
        single_string_list_of_data.clear()
        single_index_data_value = 0


Convert all the lists (date list and value list) in Pandas DataFrame, then, visualize the list created

In [8]:
import pandas as pd

# convert the 2 lists in a pandas dataframe
data = pd.DataFrame({'date': list_of_datetime,
                     'value': list_of_float})

# print out the datagram
data

,date,value
0,2020-11-08 00:10:03,2.8
1,2020-11-08 00:30:03,5.2
2,2020-11-08 00:50:05,6.6
3,2020-11-08 01:10:03,3.0
4,2020-11-08 01:30:04,1.5
...,...,...
66,2020-11-08 22:10:03,9.9
67,2020-11-08 22:30:04,1.4
68,2020-11-08 22:50:09,1.5
69,2020-11-08 23:10:02,1.6


# Draw Chart

Draw the Chart using the Bokeh library (the reason why i used this library is because it is easily customizable and it supports the callback event, in wich though, you have to write the code in JS; there is a method to convert some data from JS data kernel to Python data kernel by using IPython, but because of google colab enviroment, wich for some reason doesn't work, it is not possible. (a solution is to install and use Jupyter Notebook))

In [88]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.tools import *
from bokeh.models.widgets import DataTable, TableColumn, DateFormatter
from bokeh.layouts import row


# Configure the default output state to generate output in notebook cells when show() is called. 
output_notebook()

# ColumnDataSource is a special object, wich it is easy to share data between multiple plots and widgets
# Also the same ColumnDataSource is used to drive multiple renderers, selections of the data source are also shared.
source = ColumnDataSource(data)

# select the desired tool that can be used with the chart
chart_tools="pan,wheel_zoom,reset,hover,poly_select,box_select"
tooltip = HoverTool(
    # specify wich type of info i want to show
    tooltips=[
        ("index", "$index"),
        ("Value", "@value{0.2f}"),
        ("Date", "@date{%F}"),
    ],
    # format the date in datetime type (the default is numerical format)
    formatters={
        '@date': 'datetime'
    },
    # display a tooltip whenever the cursor is vertically in line with a data point
    mode='vline'
)

# prepare the chart with the selected options, and add the tooltip too
chart = figure(title='Data Visualization', plot_width=800, plot_height=400, y_axis_label='Values', x_axis_label='Date', x_axis_type='datetime',
           tools=chart_tools, toolbar_location='above', tooltips="")
chart.add_tools(tooltip)

# draw in the chart both line and circle, so that the obtained effect is that of linked points
symbol1 = chart.line(x='date', y='value', line_width=2, source=source)
symbol2 = chart.circle(x='date', y='value', fill_color="white", size=6, source=source)


# create table
source_table = ColumnDataSource(data=dict(
    Start_date=[],
    End_date=[],
    Max_value=[],
    Average_value=[],
    Area=[]
))

columns = [
        TableColumn(field="Start_date", title="Start_date", formatter=DateFormatter(format="%m/%d/%Y - %H:%M:%S")),
        TableColumn(field="End_date", title="End_date", formatter=DateFormatter(format="%m/%d/%Y - %H:%M:%S")),
        TableColumn(field="Max_value", title="Max_value"),
        TableColumn(field="Average_value", title="Average_value"),
        TableColumn(field="Area", title="Area")
    ]

data_table = DataTable(columns=columns, source=source_table, width=700, height=400)

# Execute the JS code when the user select any data in the chart with the box select tool
source.selected.js_on_change("indices", CustomJS(args=dict(source=source, source_table=source_table), code="""
        // get the indices of the selectd data
        var inds = cb_obj.indices


        // create various variable to store the data from the source (ColumnDataSource object)
        var d1 = source.data
        var d2 = source_table.data


        // get the first and end date (no need to any particular calculus because each data is already in order and unique)
        var start_date = new Date(d1["date"][0])
        var end_date = new Date(d1["date"][inds.length-1])


        // get max value
        var max_value
        for(var i = 0; i < inds.length; i++) {
            if(i == 0) {
              max_value = d1["value"][inds[i]]
            } else {
              if(max_value < d1["value"][inds[i]]){
                max_value = (d1["value"][inds[i]]).toFixed(2)
              }
            }
        }


        // get average value
        var avarage_value = 0.00
        for(var i = 0; i < inds.length; i++) {
            avarage_value = avarage_value + d1["value"][inds[i]]
        }
        avarage_value = (avarage_value/inds.length).toFixed(2)


        // calculate the area (by using trapezoid)(consider the value in minutes)
        var area = 0.00
        var value_list = []
        var minute_list = []
        for(var i = 0; i < inds.length; i++) {
            value_list.push(d1["value"][inds[i]])
            minute_list.push((new Date(d1["date"][inds[i]])).getMinutes())
            console.log(new Date(d1["date"][inds[i]]))
            console.log((new Date(d1["date"][inds[i]])).getMinutes())
            console.log(minute_list)
        }


        // push all the data in the source of the data_table
        d2["Start_date"].push(start_date)
        d2["End_date"].push(end_date)
        d2["Max_value"].push(max_value)
        d2["Average_value"].push(avarage_value)
        d2["Area"].push("a")


        // update the table (this function notify that something has changed, and to immediately update the values and UI)
        source_table.change.emit()
        """
))

# visualize the chart in the output
layout = row(chart, data_table)
show(layout)

In [14]:
from random import random

from bokeh.layouts import row
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import DataTable, TableColumn, DateFormatter

output_file("callback.html")

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(x=[], y=[]))

columns = [
        TableColumn(field="x", title="Date", formatter=DateFormatter()),
        TableColumn(field="y", title="Value"),
    ]

p2 = DataTable(columns=columns, source=s2)

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        var inds = cb_obj.indices;
        var d1 = s1.data;
        var d2 = s2.data;

        for (var i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)
)

layout = row(p1, p2)

show(layout)

# Other Library used to visualize data


Test of creating the chart using various libraries...


Draw everything in a chart using the altair library (i haven't used this library since it doesn't have any support for return the data selected)

In [11]:
import altair as alt

brush = alt.selection_interval(encodings=['x'])

color = alt.condition(brush, alt.value('blue'), alt.value('lightgray'))

chart = alt.Chart(data).mark_line(point={
    "filled": False,
    "fill": 'white'
    }).encode(
        y=alt.Y('value', sort=alt.EncodingSortField('value', order='descending')),
        x='date',
        color=color
).add_selection(
    brush
)

chart 

alt.Chart(...)

Draw everything in a chart using the Plotly library (i haven't used this library since it doesn't have any support for return the data selected, an exemption is DASH that is the framework built-on this library, but using this, require a few tweaks)

In [12]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=data.date, y=data.value, mode="markers+lines"))

scatter = fig.data[0]

def update_point(trace, points, selector):
  print("Sleeping")
scatter.on_click(update_point)

fig.show(renderer="colab")